# Toronto Territory Tussle (Week 3)
This notebook will be used for the capstone project in the IBM Applied Data Science Specialization on Coursera.

## 1. Download and Explore Dataset

First of all we import the _pandas_ library to load and manipulate datasets.

In [1]:
import pandas as pd # library for data analsysis

print('Pandas imported.')

Pandas imported.


In the next step, we load the table from https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M into a _pandas_ dataframe. A quick look into the page's HTML code tells us that the table is contained in the class "wikitable".

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df = pd.read_html(url, attrs={"class": "wikitable"})[0] # 0 is for the 1st table in this particular page
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


---
Rename the first column to adhere to the specifications giving in the excercise and sort by postal code:

In [3]:
df.rename(columns={"Postal Code":"PostalCode"}, inplace=True)
df.sort_values(by=["PostalCode"], inplace=True)
df.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
9,M1B,Scarborough,"Malvern, Rouge"
18,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
27,M1E,Scarborough,"Guildwood, Morningside, West Hill"
36,M1G,Scarborough,Woburn


---
Dropping all rows that do not have an assigned borough:

In [4]:
df = df[df["Borough"]!="Not assigned"].reset_index(drop=True)
df.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


---
Checking duplicate postal codes:

In [5]:
row_no = df.shape[0]
pc_no = df["PostalCode"].unique().shape[0]

print("Number of rows in dataframe:", row_no)
print("Number of unique postal codes:", pc_no)

print("There are", row_no - pc_no, "duplicate postal codes.")

Number of rows in dataframe: 103
Number of unique postal codes: 103
There are 0 duplicate postal codes.


It appears that the Wikipedia page has been updated since the creation of this week's excercise.

But as the goal of this whole course is to learn how to manipulate data with python, I created a dummy file on which I tried to complete the excercise:

In [165]:
df_dummy = pd.read_csv("dummy_data.csv")
df_dummy.head()

,PostalCode,Borough,Neighborhood
0,PC1,Borough1,Neighborhood1
1,PC1,Borough1,Neighborhood2
2,PC2,Borough2,Not assigned


Group the dataframe by postal codes and create lists with unique boroughs and neighborhoods that belong to said code.
To achieve that we use the pandas.DataFrame.groupby().agg() method with the "unique" function.

In [166]:
df_dummy = df_dummy.groupby("PostalCode").agg({"Borough": "unique", "Neighborhood": "unique"})
df_dummy.head()

,Borough,Neighborhood
PostalCode,,
PC1,[Borough1],"[Neighborhood1, Neighborhood2]"
PC2,[Borough2],[Not assigned]


As we don't want numpy.ndarrays as entries in the dataframe, let's map them to strings using the join method.
pd.DataFrame.applymap() applies a function to every element of the dataframe.

In [167]:
df_dummy = df_dummy.applymap(lambda x: ", ".join(x))
df_dummy.head()

,Borough,Neighborhood
PostalCode,,
PC1,Borough1,"Neighborhood1, Neighborhood2"
PC2,Borough2,Not assigned


Turning "PostalCode" into a column:

In [168]:
df_dummy.reset_index(inplace=True)
df_dummy

,PostalCode,Borough,Neighborhood
0,PC1,Borough1,"Neighborhood1, Neighborhood2"
1,PC2,Borough2,Not assigned


---
Now we check whether there are any remaining rows in the dataframe with not assigned neighbourhoods.

In [169]:
nA_neighbourhoods = df[df["Neighbourhood"]=="Not assigned"].shape[0]
print("There are", nA_neighbourhoods ,"entries with not assigned neighbourhoods")

There are 0 entries with not assigned neighbourhoods


There is no need to change any entries in the dataframe.

Let us once again check our dummy data:

In [170]:
df_dummy.head()

,PostalCode,Borough,Neighborhood
0,PC1,Borough1,"Neighborhood1, Neighborhood2"
1,PC2,Borough2,Not assigned


Obviously, the Neighborhood for PC2 is not assigned. Let's set it to NaN, then fill NaN values with the value of the preceding column, i.e. the corresponding borough.

Unfortunately, while the 'replace' method also supports the argument 'method="ffill"', it is currently not possible to choose the axis. Otherwise we wouldn't have to use such a roundabout way to do this.

In [173]:
df_dummy = df_dummy.replace(to_replace="Not assigned", value= np.nan).fillna(method='ffill', axis=1)
df_dummy.head()

,PostalCode,Borough,Neighborhood
0,PC1,Borough1,"Neighborhood1, Neighborhood2"
1,PC2,Borough2,Borough2


In the general case it would probably be better to perform this replacement step before merging duplicate postal codes, otherwise it might be possible to not catch all values that are not assigned.

---
According to the final part of the first excercise, we print the number of rows in the dataframe:

In [7]:
row_no = df.shape[0]
print("There are", row_no, "rows in the dataframe.")

There are 103 rows in the dataframe.


## 2. Add Geospatial Data
As the geocoder package is said to be unreliable, we download the provided file from http://cocl.us/Geospatial_data and read it into a pandas dataframe.

In [8]:
df_geoloc = pd.read_csv('Geospatial_Coordinates.csv')
df_geoloc.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Once again, we change the name of the first column.

In [9]:
df_geoloc.rename(columns={"Postal Code":"PostalCode"}, inplace=True)
df_geoloc.sort_values(by=["PostalCode"], inplace=True)
df_geoloc.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Check if the number of rows is 103. If that is not the case, there is a bug or another mistake that needs fixing.

In [10]:
if df_geoloc.shape[0] == row_no:
    print("Equal number of rows.")
else:
    print("Number of rows is not equal!")

Equal number of rows.


---
Merge both dataframes into one:

In [11]:
df=pd.merge(df, df_geoloc, on='PostalCode')
df.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [12]:
print("Number of rows:", df.shape[0])

Number of rows: 103


Merging the dataframes was successful.

## 3. Toronto Data Analysis

In the following section we perform an analysis of the neighbourhoods.
Before we start, let's import all the libraries we need.

In [13]:
import numpy as np # library to handle data in a vectorized manner

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Adding credentials to use the Foursquare API (please excuse the fact that I do not want to make my credentials public ;)):

In [159]:
with open("foursquareKeys.txt") as keyFile:
    CLIENT_ID, CLIENT_SECRET = keyFile.read().splitlines()

VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value